# 🚗 Vehicle Classification from CCTV
A complete walkthrough for classifying vehicle types from simulated CCTV-style images using a CNN.

## 📦 Step 1: Import Required Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import pickle


## 📁 Step 2: Load and Explore the Dataset

In [ ]:
df = pd.read_csv("data/vehicle_labels.csv")
df.head()


## 🧠 Step 3: Encode Labels for Training

In [ ]:
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["vehicle_type"])
with open("model/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)
df.head()


## 🖼️ Step 4: Load and Preprocess Images

In [ ]:
IMG_SIZE = (64, 64)
images = []
labels = df["label_id"].values

for path in df["image_path"]:
    img = load_img(path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    images.append(img_array)

X = np.array(images)
y = to_categorical(labels)
X.shape, y.shape


## ✂️ Step 5: Split Dataset

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


## 🏗️ Step 6: Build CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 🚀 Step 7: Train the CNN Model

In [ ]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=8, callbacks=[early_stop])


## 📊 Step 8: Evaluate the Model

In [ ]:
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=le.classes_))


## 💾 Step 9: Save the Model and Encoder

In [ ]:
os.makedirs("model", exist_ok=True)
model.save("model/vehicle_cnn_model.h5")
